In [6]:
import pandas as pd
#pd.options.plotting.backend = 'hvplot'
import numpy as np
import datetime as dt
from isoweek import Week
from time import gmtime
import os
#import matplotlib.pyplot as plt
from functools import reduce
import re
import warnings
warnings.filterwarnings('ignore')
#from pivottablejs import pivot_ui
from IPython.display import HTML
from datetime import date, timedelta 
#import nltkx
from datetime import datetime
from datetime import datetime, time
import plotly.express as px
from data_processing.kpi_generation import processing_df_ticket
#import pygwalker as pyg


#path = %pwd
path_source_affid_hubspot_ticket = '../data/Affid/Hubspot/ticket'
path_source_affid_hubspot_agent = '../data/Affid/Hubspot/agent'

files_hubspot_ticket = [file for file in os.listdir(path_source_affid_hubspot_ticket) if not file.startswith('.')] # Ignore hidden files

data_affid_hubspot_ticket = pd.DataFrame()

for file in files_hubspot_ticket:
    current_data_hubspot = pd.read_excel(path_source_affid_hubspot_ticket +"/"+ file)
    data_affid_hubspot_ticket = pd.concat([data_affid_hubspot_ticket, current_data_hubspot])

files_hubspot_agent = [file for file in os.listdir(path_source_affid_hubspot_agent) if not file.startswith('.')] # Ignore hidden files

data_affid_hubspot_agent = pd.DataFrame()

for file in files_hubspot_agent:
    current_data_hubspot_agent = pd.read_excel(path_source_affid_hubspot_agent +"/"+ file)
    data_affid_hubspot_agent = pd.concat([data_affid_hubspot_agent, current_data_hubspot_agent])




data_affid_hubspot_agent = data_affid_hubspot_agent[(data_affid_hubspot_agent["Nombre d'e-mails envoyés"] == 1) 
                                                    & (data_affid_hubspot_agent["Conversation ID"] != "(Aucune valeur)")]

data_affid_hubspot_agent = data_affid_hubspot_agent[data_affid_hubspot_agent['Source'].isin(['Chat', 'Formulaire', 'E-mail'])]


data_affid_hubspot_agent["Date d'activité"] = pd.to_datetime(data_affid_hubspot_agent["Date d'activité"])
data_affid_hubspot_agent["Semaine"] = data_affid_hubspot_agent["Date d'activité"].dt.strftime("S%Y-%V")

data_affid_hubspot_agent.sort_values(["Date d'activité"], ascending=True, inplace=True)



def processing_df_ticket(data_affid_hubspot_ticket): 
    df = data_affid_hubspot_ticket

    # Conversion des colonnes en datetime
    df['Date de création'] = pd.to_datetime(df['Date de création'], format='%d/%m/%Y %H:%M')
    df["Date de la première réponse par e-mail de l'agent"] = pd.to_datetime(df["Date de la première réponse par e-mail de l'agent"], format='%d/%m/%Y %H:%M')

    def calculate_working_hours(start, end):

        # Limites des heures ouvrées
        start_hour, end_hour = 9, 18

        # Initialisation du temps ouvré total à 0
        total_working_hours = 0

        # Boucle sur chaque jour entre start et end
        current = start
        while current < end:
            next_day = (current + pd.Timedelta(days=1)).normalize()

            # Si c'est un jour ouvré
            if current.weekday() < 5:  # 0=lundi, 4=vendredi
                # Début des heures ouvrées pour ce jour
                work_start = current.replace(hour=start_hour, minute=0, second=0)
                # Fin des heures ouvrées pour ce jour
                work_end = current.replace(hour=end_hour, minute=0, second=0)

                # Calcul des heures travaillées pour ce jour
                day_start = max(current, work_start)
                day_end = min(end, work_end, next_day)

                # Ajouter les heures de ce jour à la somme totale si elles sont dans les heures ouvrées
                if day_start < day_end:
                    total_working_hours += (day_end - day_start).total_seconds() / 3600

            # Passer au jour suivant
            current = next_day

        return total_working_hours

    # Application de la fonction pour chaque ligne
    df['working_hours'] = df.apply(lambda row: calculate_working_hours(row['Date de création'], row["Date de la première réponse par e-mail de l'agent"]), axis=1)

    def convert_hours_to_hms(hours):
        total_seconds = int(hours * 3600)
        hours, remainder = divmod(total_seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        return f"{hours:02}:{minutes:02}:{seconds:02}"

    # Application de la fonction à la colonne working_hours
    df['working_hours_hms'] = df['working_hours'].apply(convert_hours_to_hms)

    df["Semaine"] = df['Date de création'].dt.strftime("S%Y-%V")
    df["Date"] = df["Date de création"].dt.date
    df["Mois"] = df['Date de création'].dt.strftime("S%Y-%m")

    #df['Nombre_ticket_client'] = df.apply(
    #lambda x: 1 if x['Statut du ticket'] not in ['Résolu', 'Clos'] and x['Source'] in ['E-mail', 'Chat', 'Formulaire'] else 0,
    #axis=1
    #)

    df['Nombre_ticket_client'] = df.apply(
    lambda x: 1 if x['Statut du ticket'] and x['Source'] in ['E-mail', 'Chat', 'Formulaire'] else 0,
    axis=1
    )

    df['Nombre_ticket_telephone'] = df.apply(
    lambda x: 1 if x['Statut du ticket'] and x['Source'] in ['Téléphone'] else 0,
    axis=1
    )
    
    #df['Ticket_ouvert'] = df['Délai de traitement (HH:mm:ss)'].apply(lambda x: 1 if pd.isna(x) else 0)


    return df



def repartition_activite_agent(data_affid_hubspot_agent): 

    df = data_affid_hubspot_agent.groupby(["Propriétaire du ticket"]).agg({"Nombre d'e-mails envoyés":"sum", 
                                                                                "Ticket ID":"nunique"}).reset_index()


    fig = px.pie(
        df, 
        names="Propriétaire du ticket", 
        values="Nombre d'e-mails envoyés", 
        title="Répartition des e-mails envoyés par propriétaire de ticket"
    )

    return fig

def repartition_tickets_agent_semaine(df): 
    
    df = df[df['Propriétaire du ticket'].isin(['Archimède KESSI', 
                                'HUMBLOT NASSUF', 
                                'FREDERIC SAUVAN', 
                                'Christophe Brichet', 
                                'Pierre Goupillon', 
                                'Olivier Sainte-Rose'])].groupby(["Semaine","Propriétaire du ticket"]).agg({"Nombre_ticket_client":"sum"}).reset_index()

    ordre_semaines = sorted(df['Semaine'].unique(), key=lambda x: (int(x[1:5]), int(x[6:])))

    fig = px.bar(
        df,
        x="Semaine",
        y="Nombre_ticket_client",
        color="Propriétaire du ticket",
        title="Heures de travail par pipeline et par semaine",
        #labels={"working_hours": "Heures de travail", "Semaine": "Semaine"},
    )

    fig.update_layout(
        xaxis=dict(categoryorder="array", categoryarray=ordre_semaines),  # Définit l'ordre des catégories
        yaxis_title="Appels",
        #yaxis=dict(range=[0, 200])  # Ajuste la plage de l'axe y entre 0 et 100
    )

    return fig


def activite_ticket_source_client(df): 

    df = df[df['Source'].isin(['Chat', 'E-mail', 'Formulaire'])].groupby(['Semaine', 'Pipeline'])['Ticket ID'].nunique().reset_index()

    fig = px.bar(
        df,
        x="Semaine",
        y="Ticket ID",
        color="Pipeline",
        title="Heures de travail par pipeline et par semaine",
        labels={"working_hours": "Heures de travail", "Semaine": "Semaine"},
    )
    ordre_semaines = sorted(df['Semaine'].unique(), key=lambda x: (int(x[1:5]), int(x[6:])))
    
    fig.update_layout(
        xaxis=dict(categoryorder="array", categoryarray=ordre_semaines),  # Définit l'ordre des catégories
        #yaxis_title="Appels",
        #yaxis=dict(range=[0, 200])  # Ajuste la plage de l'axe y entre 0 et 100
    )
    return fig


def activite_ticket_source_pipeline(df): 

    df = df[df['Source'].isin(['Chat', 'E-mail', 'Formulaire'])].groupby(['Pipeline'])['Ticket ID'].nunique().reset_index()


    fig = px.pie(
        df, 
        names="Pipeline", 
        values="Ticket ID", 
        title="Répartition des e-mails envoyés par propriétaire de ticket"
    )


    return fig



ModuleNotFoundError: No module named 'data_processing'

In [30]:
df_ticket['Propriétaire du ticket'].value_counts()

Propriétaire du ticket
Archimède KESSI                  1605
Emilie Gest                      1357
HUMBLOT NASSUF                   1237
FREDERIC SAUVAN                  1067
Sandrine Sauvage                 1048
Christophe Brichet                797
Melinda Marmin                    749
Olivier Sainte-Rose               617
Morgane VANDENBUSSCHE             529
Pierre Goupillon                  513
Vincent LE QUENNEC                260
Sophie FONTAINE                    99
LESLIE DE MARCH POUTREL            68
Catherine Goupillon                41
nathalie pottiez                   40
Eveline BISCARA                    39
Sébastien HUREL                    32
THIERRY CAROFF                     29
Céline Crendal                     18
Cédric PRADALET                    16
Vincent Gourvat                    15
Olga EKIMTCEVA                     13
Nicolas Razafindramena             12
cremir lebo                        10
Soufiane Ghanim                    10
Nathalie Pottiez           

In [28]:
df_ticket[(df_ticket['Propriétaire du ticket'] == 'Morgane VANDENBUSSCHE')
          & (df_ticket['Pipeline'].isin(['SSI', 'SSIA']))]['Ticket ID'].nunique()    

434

In [27]:
df_ticket[(df_ticket['Propriétaire du ticket'] == 'Morgane VANDENBUSSCHE')
          & (df_ticket['Pipeline'].isin(['SSI', 'SSIA'])) 
          & (df_ticket['Passé par le support N2'] == 'Oui')]['Ticket ID'].nunique()

103

In [35]:
df_ticket[(df_ticket['Propriétaire du ticket'] == 'Emilie Gest')
          & (df_ticket['Pipeline'].isin(['SSI', 'SSIA', 'SPSA']))]['Ticket ID'].nunique() 

1347

In [36]:
df_ticket[(df_ticket['Propriétaire du ticket'] == 'Emilie Gest')
          & (df_ticket['Pipeline'].isin(['SSI', 'SSIA', 'SPSA'])) 
          & (df_ticket['Passé par le support N2'] == 'Oui')]['Ticket ID'].nunique()

58

In [3]:
entrant = df_support.groupby(['Semaine','Date'], as_index=False).agg({'Number': 'nunique'})
entrant = entrant.rename(columns={'Number': 'Entrant'})
entrant = entrant.groupby('Semaine')['Entrant'].sum()


# Filtrage des tickets selon la source, puis agrégation
ticket = df_ticket[
(df_ticket['Source'].isin(['Chat', 'E-mail', 'Formulaire'])) &
(df_ticket['Pipeline'].isin(['SSI', 'SSIA']))
].groupby('Semaine', as_index=False)['Ticket ID'].nunique()

ticket.rename(columns={'Ticket ID': 'Nb_Tickets'}, inplace=True)


# Fusionner les deux pour garantir la cohérence des semaines
df_merged = pd.merge(entrant, ticket, on='Semaine', how='outer').fillna(0)

In [5]:
df_merged['Entrant'].sum() / (df_merged['Nb_Tickets'].sum() + df_merged['Entrant'].sum() )

0.8728503965071728

In [18]:
df_ticket.groupby('Propriétaire du ticket')['Ticket ID'].nunique().mean() / df_ticket.groupby('Propriétaire du ticket')['Date'].nunique().mean()

3.6851011143210894

In [98]:
df_support_agent = df_support[df_support['UserName'] == 'Archimede KESSI']

df_support_agent.groupby('Semaine')['Entrant_connect'].sum().mean() + df_support_agent.groupby('Semaine')['Sortant_connect'].sum().mean()

71.78333333333333

In [147]:
df_support

,line,Semaine,Date,Jour,Heure,direction,LastState,ScenarioName,StartTime,HangupTime,time (TZ offset incl.),TotalDuration,InCallDuration,FromNumber,ToNumber,UserName,Tags,IVR Branch,Number,Count,Entrant_connect,Entrant,Sortant_connect,Taux_de_service,Mois,Effectif,Logiciel
0,technique,S2023-49,2023-12-08,Friday,13,inbound,no,no_available_agent,2023-12-08,1900-01-01 13:14:55,1900-01-01 13:14:55,162.0,0.0,33621516371,3.318973e+10,NaN,nan,,33621516371,1,0,1,0,0.0,2023-12,6.0,Inconnu
1,technique,S2023-49,2023-12-08,Friday,13,inbound,no,no_available_agent,2023-12-08,1900-01-01 13:08:05,1900-01-01 13:08:05,152.0,0.0,33661133885,3.318973e+10,NaN,nan,,33661133885,1,0,1,0,0.0,2023-12,6.0,Inconnu
2,technique,S2023-49,2023-12-08,Friday,13,inbound,no,no_available_agent,2023-12-08,1900-01-01 13:05:06,1900-01-01 13:05:06,171.0,0.0,33661133885,3.318973e+10,NaN,nan,,33661133885,1,0,1,0,0.0,2023-12,6.0,Inconnu
3,technique,S2023-49,2023-12-08,Friday,12,inbound,yes,NaN,2023-12-08,1900-01-01 13:18:55,1900-01-01 12:59:47,1201.0,1148.0,33609541997,3.318973e+10,Mourad HUMBLOT,nan,,33609541997,1,1,1,0,1.0,2023-12,6.0,Inconnu
4,technique,S2023-49,2023-12-08,Friday,13,inbound,no,agents_did_not_answer,2023-12-08,1900-01-01 13:17:47,1900-01-01 13:17:47,220.0,0.0,33621516371,3.318973e+10,NaN,nan,,33621516371,1,0,1,0,0.0,2023-12,6.0,Inconnu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37625,standard,S2025-13,2025-03-25,Tuesday,17,outbound,yes,NaN,2025-03-25,1900-01-01 17:57:23,1900-01-01 17:51:03,399.0,380.0,33187662300,3.360346e+10,Mourad HUMBLOT,nan,NaN,33603458520.0,1,0,0,1,NaN,2025-03,8.0,Inconnu
37626,standard,S2025-13,2025-03-25,Tuesday,17,outbound,yes,NaN,2025-03-25,1900-01-01 17:33:39,1900-01-01 17:33:00,57.0,39.0,33187662300,3.360346e+10,Mourad HUMBLOT,nan,NaN,33603458520.0,1,0,0,0,NaN,2025-03,8.0,Inconnu
37627,standard,S2025-13,2025-03-27,Thursday,15,outbound,yes,NaN,2025-03-27,1900-01-01 15:30:32,1900-01-01 15:17:05,818.0,807.0,33187662300,3.360320e+10,Christophe Brichet,nan,NaN,33603202223.0,1,0,0,1,NaN,2025-03,6.0,Inconnu
37628,standard,S2025-13,2025-03-25,Tuesday,15,outbound,yes,NaN,2025-03-25,1900-01-01 15:28:59,1900-01-01 15:16:49,743.0,730.0,33187662300,3.360769e+10,Archimede KESSI,nan,NaN,33607694505.0,1,0,0,1,NaN,2025-03,8.0,Inconnu


In [93]:
df_filtered_pipeline = df_ticket[
        (df_ticket['Pipeline'].isin(['SSIA', 'SSI', 'SPSA'])) & (df_ticket['Propriétaire du ticket'].isin(['Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose', 'Morgane VANDENBUSSCHE'])) &
        ((df_ticket['Source'].isin(['Chat', 'E-mail', 'Formulaire'])) | pd.isna(df_ticket['Source']))
    ]

df_filtered_pipeline.groupby('Propriétaire du ticket')['Ticket ID'].nunique().mean() / df_filtered_pipeline.Semaine.nunique()




8.494186046511627

In [131]:
def compute_ticket_metrics(df_tickets, df_support, agent_ticket, agent_support):
    support_agents = ['Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose', 'Archimede KESSI', 'Morgane VANDENBUSSCHE']
    armatis_agents = ['Emilie Gest', 'Sandrine Sauvage', 'Melinda Marmin', 'Morgane VANDENBUSSCHE']
    agent_all = ['Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose', 'Morgane VANDENBUSSCHE']

    selected_agent_ticket = [agent_ticket] if isinstance(agent_ticket, str) else agent_ticket

    selected_agent_appel = [agent_support] if isinstance(agent_support, str) else agent_support

    # Filtrage des tickets pertinents
    df_filtered_agent = df_tickets[
        (df_tickets['Propriétaire du ticket'].isin(selected_agent_ticket)) &
        ((df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire'])) | pd.isna(df_tickets['Source']))
    ]

    df_filtered_pipeline = df_tickets[
        (df_tickets['Pipeline'].isin(['SSIA', 'SSI', 'SPSA'])) & (df_tickets['Propriétaire du ticket'].isin(agent_all)) &
        ((df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire'])) | pd.isna(df_tickets['Source']))
    ]

    # Filtrage des données support
    if any(agent in support_agents for agent in selected_agent_appel):
        df_support_filtered = df_support[df_support['UserName'].isin(support_agents)]
    elif any(agent in armatis_agents for agent in selected_agent_appel):
        df_support_filtered = df_support[df_support['UserName'].isin(armatis_agents)]
    else:
        df_support_filtered = df_support  # Si aucun agent ne correspond, prendre tous les agents

    
    df_support_agent = df_support[df_support['UserName'].isin(selected_agent_appel)]


    # Moyenne des appels entrants : Total des appels entrants par utilisateur
    total_moyenne_des_appels_traites = (df_support_filtered.groupby('UserName')['Entrant_connect'].sum().mean() + df_support_filtered.groupby('UserName')['Sortant_connect'].sum().mean())/ df_support_filtered['Semaine'].nunique()

    # Moyenne des tickets traités par jour par agent
    total_moyenne_des_tickets_traites = df_filtered_pipeline.groupby('Propriétaire du ticket')['Ticket ID'].nunique().mean() / df_filtered_pipeline.Semaine.nunique()

    agent_moyenne_des_appels_traites = df_support_agent.groupby('Semaine')['Entrant_connect'].sum().mean() + df_support_agent.groupby('Semaine')['Sortant_connect'].sum().mean()


    agent_moyenne_des_tickets_traites = df_filtered_agent['Ticket ID'].nunique() / df_filtered_agent['Semaine'].nunique()

    Total_des_tickets = df_filtered_pipeline['Ticket ID'].nunique() / df_filtered_pipeline['Semaine'].nunique()

    if df_tickets.empty:
        return 0, 0, 0, 0  # Si le DataFrame des tickets est vide, retourner des valeurs nulles.

    # Regroupement par date pour les tickets traités
    tickets_per_day = df_filtered_agent.groupby("Date")['Ticket ID'].nunique()

    tickets_total = tickets_per_day.sum()

    # Moyenne journalière totale des tickets traités
    #daily_avg_total = tickets_per_day.mean()

    # Moyenne journalière des tickets traités par agent
    #nb_agents = len(selected_agents) if selected_agents else 1
    #daily_avg_per_agent = daily_avg_total / nb_agents

    return round(Total_des_tickets, 2), round(agent_moyenne_des_tickets_traites, 2), round(total_moyenne_des_appels_traites, 2), round(total_moyenne_des_tickets_traites, 2), round(agent_moyenne_des_appels_traites, 2) 


In [132]:
compute_ticket_metrics(df_ticket, df_support, 'Archimède KESSI', 'Archimede KESSI')

(33.98, 17.68, 60.16, 8.49, 71.78)

In [171]:
def compute_ticket_metrics(df_tickets, df_support, agent_ticket, agent_support):
    # Définir les listes d'agents sans doublons
    agents_n1 = ['Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose', 
                'Mourad HUMBLOT', 'Archimede KESSI', 'Morgane Vandenbussche', 'Morgane VANDENBUSSCHE']
    agents_all = ['Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose', 'Morgane VANDENBUSSCHE', 
                  'Emilie Gest', 'Sandrine Sauvage', 'Melinda Marmin']  # Liste combinée de tous les agents

    # Gérer les agents sélectionnés
    selected_agent_ticket = [agent_ticket] if isinstance(agent_ticket, str) else agent_ticket
    selected_agent_appel = [agent_support] if isinstance(agent_support, str) else agent_support

    # Filtrage des tickets pertinents
    df_filtered_agent = df_tickets[
        (df_tickets['Propriétaire du ticket'].isin(selected_agent_ticket)) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]
    df_filtered_pipeline = df_tickets[
        (df_tickets['Pipeline'].isin(['SSIA', 'SSI', 'SPSA'])) &
        (df_tickets['Propriétaire du ticket'].isin(agents_n1)) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]

    # Filtrage des données support selon les agents
    if any(agent in agents_n1 for agent in selected_agent_appel):
        df_support_filtered = df_support[df_support['UserName'].isin(agents_n1)]
    else:
        df_support_filtered = df_support 
        
    total_support_total = df_support[
        (df_support['line'].isin(['technique', 'armatistechnique'])) &
        (df_support['direction'].isin(['inbound']))
    ]
    total_support_total = total_support_total.groupby('Semaine')['Count'].sum().mean()

    total_ticket_total = df_tickets[
        (df_tickets['Pipeline'].isin(['SSIA', 'SSI', 'SPSA'])) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]

    total_ticket_total = total_ticket_total.groupby('Semaine')['Ticket ID'].nunique().mean()

    # Filtrage des données pour les agents spécifiques
    df_support_agent = df_support[df_support['UserName'].isin(selected_agent_appel)]

    # Calcul des moyennes
    total_moyenne_des_appels_traites = (
        (df_support_filtered.groupby('UserName')['Entrant_connect'].sum().mean() +
         df_support_filtered.groupby('UserName')['Sortant_connect'].sum().mean()) /
        df_support_filtered['Semaine'].nunique()
    )

    total_moyenne_des_tickets_traites = (
        df_filtered_pipeline.groupby('Propriétaire du ticket')['Ticket ID'].nunique().mean() /
        df_filtered_pipeline['Semaine'].nunique()
    )

    agent_moyenne_des_appels_traites = (
        df_support_agent.groupby('Semaine')['Entrant_connect'].sum().mean() +
        df_support_agent.groupby('Semaine')['Sortant_connect'].sum().mean()
    )

    agent_moyenne_des_tickets_traites = (
        df_filtered_agent['Ticket ID'].nunique() / df_filtered_agent['Semaine'].nunique()
    )

    Total_des_tickets = (
        df_filtered_pipeline['Ticket ID'].nunique() / df_filtered_pipeline['Semaine'].nunique()
    )

    Total_des_appels = (
        df_filtered_pipeline['Ticket ID'].nunique() / df_filtered_pipeline['Semaine'].nunique()
    )

    if df_tickets.empty:
        return 0, 0, 0, 0  # Si le DataFrame des tickets est vide, retourner des valeurs nulles.

    # Calcul des tickets par jour
    tickets_per_day = df_filtered_agent.groupby("Date")['Ticket ID'].nunique()

    tickets_total = tickets_per_day.sum()

    return (
        int(Total_des_tickets),
        int(total_support_total),
        int(total_ticket_total),
        int(total_moyenne_des_appels_traites),
        int(total_moyenne_des_tickets_traites),
        int(agent_moyenne_des_tickets_traites),
        int(agent_moyenne_des_appels_traites)
        
    )


In [279]:
def compute_ticket_appels_metrics(df_tickets, df_support, agent_ticket, agent_support):
    # Définir les listes d'agents sans doublons
    agents_n1 = list(set([
        'Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose',
        'Mourad HUMBLOT', 'Archimede KESSI', 'Morgane Vandenbussche', 'Morgane VANDENBUSSCHE'
    ]))
    
    agents_all = [
        'Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose', 'Morgane VANDENBUSSCHE',
        'Emilie Gest', 'Sandrine Sauvage', 'Melinda Marmin'
    ]

    # Normaliser les entrées : agent ou liste d'agents
    selected_agent_ticket = [agent_ticket] if isinstance(agent_ticket, str) else agent_ticket
    selected_agent_support = [agent_support] if isinstance(agent_support, str) else agent_support

    # -- FILTRAGE DES TICKETS --

    # Tickets de l'agent sélectionné
    df_filtered_agent = df_tickets[
        (df_tickets['Propriétaire du ticket'].isin(selected_agent_ticket)) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]

    # Tickets tous agents niveau 1
    df_filtered_pipeline = df_tickets[
        (df_tickets['Pipeline'].isin(['SSIA', 'SSI', 'SPSA'])) &
        (df_tickets['Propriétaire du ticket'].isin(agents_n1)) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]

    # -- FILTRAGE SUPPORT --

    # Support pour agents sélectionnés
    df_support_agent = df_support[df_support['UserName'].isin(selected_agent_support)]

    # Support filtré pour tous les agents N1 ou global
    df_support_filtered = (
        df_support[df_support['UserName'].isin(agents_n1)]
        if any(agent in agents_n1 for agent in selected_agent_support)
        else df_support
    )

    # Support global pour toutes lignes techniques
    df_support_total = df_support[
        (df_support['line'].isin(['technique', 'armatistechnique', 'standard']))
    ]
    moyenne_appels_entrant_total = df_support_total[df_support_total['direction'] == 'inbound'].groupby('Date')['Count'].sum().mean()

    # Tickets globaux pour calcul du volume total hebdomadaire
    df_tickets_total = df_tickets[
        (df_tickets['Pipeline'].isin(['SSIA', 'SSI'])) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]
    moyenne_tickets_total = df_tickets_total.groupby(['Date'])['Ticket ID'].nunique().mean()

    # -- CALCUL DES MOYENNES --

    # Moyenne d'appels traités globalement
    moyenne_appels_traite_global = (
        df_support_total.groupby(['Date'])[['Entrant_connect', 'Sortant_connect']].sum().mean().mean()
    )

    # Moyenne de tickets traités globalement
    moyenne_tickets_traite_global = (
        df_filtered_pipeline.groupby(['Date','Propriétaire du ticket'])['Ticket ID'].nunique().mean()
    )

    # Moyenne d'appels traités par l'agent
    moyenne_appels_traite_agent = (
        df_support_agent.groupby('Date')[['Entrant_connect', 'Sortant_connect']].sum().sum(axis=1).mean()
    )

    # Moyenne de tickets traités par l'agent
    moyenne_tickets_traite_agent = (
        df_filtered_agent['Ticket ID'].nunique() / df_filtered_agent['Date'].nunique()
        if df_filtered_agent['Date'].nunique() > 0 else 0
    )

    # Moyenne totale hebdomadaire des tickets (peut être redondante avec moyenne_tickets_traite_global)
    total_tickets_moyenne = (
        df_filtered_pipeline.groupby(['Date'])['Ticket ID'].nunique().mean()
        if df_filtered_pipeline['Date'].nunique() > 0 else 0
    )

    # Sécurité si dataframe vide
    if df_tickets.empty:
        return 0, 0, 0, 0, 0, 0, 0

    # -- RETOUR --

    return (
        int(total_tickets_moyenne), #ok
        int(moyenne_appels_entrant_total), #ok
        int(moyenne_tickets_total), #ok
        int(moyenne_appels_traite_global), #ok
        round(moyenne_tickets_traite_global,2),
        round(moyenne_tickets_traite_agent,2),
        int(moyenne_appels_traite_agent)
    )


In [278]:
# Liste normalisée des agents N1 (sans doublons ni variations d’écriture)
agents_n1 = [
        'Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose',
        'Mourad HUMBLOT', 'Archimede KESSI', 'Morgane Vandenbussche', 'Morgane VANDENBUSSCHE'
    ]

# Initialiser une liste pour stocker les résultats
rows = []

# Boucle sur chaque agent pour calculer ses indicateurs
for agent in agents_n1:
    (
        total_tickets_moyenne,
        moyenne_appels_entrant_total,
        moyenne_tickets_total,
        moyenne_appels_traite_global,
        moyenne_tickets_traite_global,
        moyenne_tickets_traite_agent,
        moyenne_appels_traite_agent
    ) = compute_ticket_appels_metrics(df_ticket, df_support, agent, agent)

    rows.append({
        'Agent': agent,
        'Tickets hebdo (global)': total_tickets_moyenne,
        'Appels entrants (total)': moyenne_appels_entrant_total,
        'Tickets hebdo (total)': moyenne_tickets_total,
        'Appels traités (global)': moyenne_appels_traite_global,
        'Tickets traités (global)': moyenne_tickets_traite_global,
        'Tickets traités (agent)': moyenne_tickets_traite_agent,
        'Appels traités (agent)': moyenne_appels_traite_agent
    })

# Construction du DataFrame final
df_kpi_agents = pd.DataFrame(rows)

# Affichage
df_kpi_agents


ValueError: cannot convert float NaN to integer

In [256]:
df_filtered_pipeline = df_ticket[
        (df_ticket['Pipeline'].isin(['SSIA', 'SSI', 'SPSA'])) &
        (df_ticket['Propriétaire du ticket'].isin([
        'Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose',
        'Mourad HUMBLOT', 'Archimede KESSI', 'Morgane Vandenbussche', 'Morgane VANDENBUSSCHE'
    ])) &
        (df_ticket['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_ticket['Source']))
    ]

df_filtered_pipeline.groupby(['Date','Propriétaire du ticket'])['Ticket ID'].nunique().mean()

2.5659050966608086

In [254]:
df_support.groupby(['Date', 'UserName'])

In [234]:
df_support.line.value_counts()

line
technique           26603
standard            10226
armatistechnique      799
adv                     2
Name: count, dtype: int64

In [146]:
pourcentage_appels = 90 / (90 + 8)
pourcentage_tickets = 8 / (90 + 8)
pourcentage_activité_tickets = 8 / 33

print(pourcentage_appels)
print(pourcentage_tickets)
print(pourcentage_activité_tickets)

0.9183673469387755
0.08163265306122448
0.24242424242424243


In [165]:
total_support_total = df_support[
        (df_support['line'].isin(['technique', 'armatistechnique'])) &
        (df_support['direction'].isin(['inbound']))
    ]

total_support_total = total_support_total.groupby('Semaine')['Number'].nunique().mean()

In [166]:
total_support_total

161.40625

In [137]:
df_support_filtered = df_support[df_support['UserName'].isin(['Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose', 
                                                              'Mourad HUMBLOT', 'Archimede KESSI', 'Morgane Vandenbussche', 'Morgane VANDENBUSSCHE'])]

total_moyenne_des_appels_traites = (
        (df_support_filtered.groupby('UserName')['Entrant_connect'].sum().mean() +
         df_support_filtered.groupby('UserName')['Sortant_connect'].sum().mean()) /
        df_support_filtered['Semaine'].nunique()
    )

In [136]:
df_support.UserName.value_counts()

UserName
Mourad HUMBLOT           8510
Archimede KESSI          5863
Olivier Sainte-Rose      5614
[No associated user]     5083
Christophe Brichet       2962
Pierre GOUPILLON         2205
Frederic SAUVAN           635
Emilie GEST               180
Morgane Vandenbussche     179
Melinda Marmin            157
Sandrine Sauvage          147
Anais Agent 1              26
Armatis Agent 3             7
Celine Crendal              6
Armatis Agent 2             5
Armatis Agent 4             5
Remi Agent 2                3
Mathilde Agent 3            3
Eveline BISCARA             2
Agent Armatis 7             1
C√©line Agent 4             1
Name: count, dtype: int64

In [286]:
def compute_ticket_appels_metrics(df_tickets, df_support, agent):
    # Définir les listes d'agents sans doublons
    agents_n1 = list(set([
        'Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose',
        'Mourad HUMBLOT', 'Archimede KESSI', 'Morgane Vandenbussche', 'Morgane VANDENBUSSCHE'
    ]))

    # Traitement spécial si agent == "agents_all"
    if agent == "agents_all":
        selected_agent = agents_n1
    else:
        selected_agent = [agent] if isinstance(agent, str) else agent

    agents_support = [
        'Archimède KESSI', 'HUMBLOT NASSUF', 'Olivier Sainte-Rose'
    ]
    agents_armatis = [
        'Morgane VANDENBUSSCHE', 'Emilie Gest', 'Sandrine Sauvage', 'Melinda Marmin'
    ]

    # Filtrage df_support selon le groupe
    if agent == "agents_all":
        df_support = df_support[df_support['UserName'].isin(agents_n1)]
    elif any(a in agents_support for a in selected_agent):
        df_support = df_support[df_support['UserName'].isin(["Mourad HUMBLOT", 'Olivier Sainte-Rose', 'Archimede KESSI'])]
    elif any(a in agents_armatis for a in selected_agent):
        df_support = df_support[df_support['UserName'].isin(["Emilie GEST", 'Morgane Vandenbussche', 'Melinda Marmin', 'Sandrine Sauvage', 'Celine Crendal'])]

    # Métriques support
    Nombre_appel_jour_agent = df_support['UserName'].nunique()
    Entrant = df_support.groupby('Date').agg({'Entrant': 'sum'}).mean().values[0].astype(int)
    Numero_unique = df_support[df_support['direction'] == 'inbound'].groupby('Date').agg({'Number': 'nunique'}).mean().values[0].astype(int)
    temps_moy_appel = df_support[df_support['InCallDuration'] > 0].InCallDuration.mean()

    # -- FILTRAGE DES TICKETS --

    df_filtered_agent = df_tickets[
        (df_tickets['Propriétaire du ticket'].isin(selected_agent)) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]

    df_filtered_pipeline = df_tickets[
        (df_tickets['Pipeline'].isin(['SSIA', 'SSI', 'SPSA'])) &
        (df_tickets['Propriétaire du ticket'].isin(agents_n1)) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]

    # -- FILTRAGE SUPPORT SUPPLEMENTAIRE --

    df_support_agent = df_support[df_support['UserName'].isin(selected_agent)]

    df_support_filtered = (
        df_support[df_support['UserName'].isin(agents_n1)]
        if any(a in agents_n1 for a in selected_agent)
        else df_support
    )

    df_support_total = df_support[
        df_support['line'].isin(['technique', 'armatistechnique', 'standard'])
    ]

    moyenne_appels_entrant_total = df_support_total[df_support_total['direction'] == 'inbound'].groupby('Date')['Count'].sum().mean()

    df_tickets_total = df_tickets[
        (df_tickets['Pipeline'].isin(['SSIA', 'SSI'])) &
        (df_tickets['Source'].isin(['Chat', 'E-mail', 'Formulaire']) | pd.isna(df_tickets['Source']))
    ]
    moyenne_tickets_total = df_tickets_total.groupby(['Date'])['Ticket ID'].nunique().mean()

    moyenne_appels_traite_global = df_support_total.groupby(['Date'])[['Entrant_connect', 'Sortant_connect']].sum().mean().mean()

    moyenne_tickets_traite_global = df_filtered_pipeline.groupby(['Date','Propriétaire du ticket'])['Ticket ID'].nunique().mean()

    moyenne_appels_traite_agent = df_support_agent.groupby('Date')[['Entrant_connect', 'Sortant_connect']].sum().sum(axis=1).mean()

    moyenne_appels_entrant_agent = df_support_agent.groupby('Date')[['Entrant_connect']].sum().sum(axis=1).mean()

    moyenne_tickets_traite_agent = (
        df_filtered_agent['Ticket ID'].nunique() / df_filtered_agent['Date'].nunique()
        if df_filtered_agent['Date'].nunique() > 0 else 0
    )

    total_tickets_moyenne = (
        df_filtered_pipeline.groupby(['Date'])['Ticket ID'].nunique().mean()
        if df_filtered_pipeline['Date'].nunique() > 0 else 0
    )

    if df_tickets.empty:
        return 0, 0, 0, 0, 0, 0, 0

    return (
        int(total_tickets_moyenne),
        int(moyenne_appels_entrant_total),
        int(moyenne_tickets_total),
        int(moyenne_appels_traite_global),
        round(moyenne_tickets_traite_global, 2),
        round(moyenne_tickets_traite_agent, 2),
        int(moyenne_appels_traite_agent), 
        int(moyenne_appels_entrant_agent)
    )



def df_compute_ticket_appels_metrics(df_ticket, df_support):
    rows = []

    agents_n1 = [
        'Olivier Sainte-Rose',
            ]

    # Boucle sur chaque agent pour calculer ses indicateurs
    for agent in agents_n1:
        (
            total_tickets_moyenne,
            moyenne_appels_entrant_total,
            moyenne_tickets_total,
            moyenne_appels_traite_global,
            moyenne_tickets_traite_global,
            moyenne_tickets_traite_agent,
            moyenne_appels_traite_agent,
            moyenne_appels_entrant_agent
        ) = compute_ticket_appels_metrics(df_ticket, df_support, agent)

        rows.append({
            'Agent': agent,
            'Tickets hebdo (global)': total_tickets_moyenne,
            'Appels entrants (total)': moyenne_appels_entrant_total,
            'Tickets hebdo (total)': moyenne_tickets_total,
            'Appels traités (global)': moyenne_appels_traite_global,
            'Tickets traités (global)': moyenne_tickets_traite_global,
            'Tickets traités (agent)': moyenne_tickets_traite_agent,
            'Appels traités (agent)': moyenne_appels_traite_agent,
            'Appels entrant (agent)' : moyenne_appels_entrant_agent
        })

    # Construction du DataFrame final
    df_kpi_agents = pd.DataFrame(rows)

    return df_kpi_agents

In [287]:
df_compute_ticket_appels_metrics(df_ticket, df_support)

,Agent,Tickets hebdo (global),Appels entrants (total),Tickets hebdo (total),Appels traités (global),Tickets traités (global),Tickets traités (agent),Appels traités (agent),Appels entrant (agent)
0,Olivier Sainte-Rose,6,23,10,21,2.57,1.99,12,3


In [10]:
pd.set_option('display.max_columns', None)

In [11]:
df

,Ticket ID,Nom du ticket,Propriétaire du ticket,Associated Contact,Pipeline,Statut du ticket,Date de création,Date de fermeture,Source,Date de la dernière activité,Délai de traitement (HH:mm:ss),Durée de la première réponse par e-mail de l'agent (HH:mm:ss),Formulaire SPSA,Nom du PS,Passé par le support N2,Commentaire de la dernière enquête CES,Date de la première réponse par e-mail de l'agent,Évaluation de le dernière enquête CES,Catégorie,Lecteur,Statut de l'accord de niveau de service sur le temps de première réponse,Sujet de la demande,Type du canal d'origine,Associated Conversation,Description du ticket,Détail source de la fiche d'informations 1,Détail source de la fiche d'informations 2,Détail source de la fiche d'informations 3,Libellés de tickets,Résolution,Résumé,Associated Company,Date de la dernière réponse du client,Temps de fermeture (HH:mm:ss),Associated Note,Date de l'activité suivante,Date de la dernière modification,Date du dernier contact,Nombre de prises de contact,Date de la dernière réponse,Dernier message du visiteur,Mis à jour par l'ID utilisateur,Temps de première réponse dans les heures SLA (HH:mm:ss),Associated Contact IDs,Associated Conversation IDs,Associated Company IDs,Associated Note IDs,working_hours,working_hours_hms,Semaine,Date,Mois,Nombre_ticket_client,Nombre_ticket_telephone
0,102101477584,- Docteur Eric Guerber,Sandrine Sauvage,Eric Guerber (eric.guerber@wanadoo.fr),ADV,Résolu,2025-03-06 21:49:00,2025-03-17 15:14:00,NaN,2025-03-10 14:37:00,257:24:14,88:47:24,NaN,NaN,NaN,NaN,2025-03-10 14:37:00,NaN,"ADV (contrats, résiliation, commerce)",NaN,NaN,Suivi de ma commande,E-mail,10635461844,"Bonjour à tous,\n\nCe client est un client his...",Changer la phase de la transaction après la si...,NaN,NaN,NaN,NaN,NaN,Docteur Eric Guerber,2025-02-06 14:18:00,257:24:14,NaN,NaT,2025-03-17 15:14:00,2025-03-10 14:37:00,60.0,2025-03-17 15:14:00,False,Sandrine Sauvage,NaN,191360107706,539691580606,5173392881,NaN,14.616667,14:37:00,S2025-10,2025-03-06,S2025-03,0,0
1,100719469815,- Docteur Eric Guerber,Sandrine Sauvage,Eric Guerber (eric.guerber@wanadoo.fr),ADV,Résolu,2025-03-05 12:35:00,2025-03-06 16:47:00,NaN,2025-03-10 08:00:00,28:12:00,28:07:14,NaN,NaN,NaN,NaN,2025-03-06 16:42:00,NaN,"ADV (contrats, résiliation, commerce)",NaN,NaN,Suivi de ma commande,E-mail,10586440910,"Bonjour,\n\nSuite de la commande du docteur GU...",Changer la phase de la transaction après la si...,NaN,NaN,NaN,NaN,NaN,Docteur Eric Guerber,2025-02-06 14:18:00,28:12:00,NaN,NaT,2025-03-10 08:02:00,2025-03-06 16:42:00,54.0,2025-03-06 16:42:00,False,Sandrine Sauvage,NaN,191360107706,536980797666,5173392881,NaN,13.116667,13:07:00,S2025-10,2025-03-05,S2025-03,0,0
2,99818804438,Demande d'information Stellair PRIUM-4 William...,Archimède KESSI,William Boucenna (william.boucenna@gmail.com),SSIA,Résolu,2025-03-03 10:54:00,2025-03-03 10:54:00,Téléphone,2025-03-03 10:54:00,00:00:53,00:00:30,NaN,NaN,NaN,NaN,2025-03-03 10:54:00,NaN,PbConnexion,PRIUM-4,Contrat-cadre conclu dans les temps,Demande d'information Stellair,E-mail,10513649887,Ralentissement Stellair,NaN,NaN,NaN,Aircall Tickets,NaN,NaN,william.boucenna@gmail.com,2025-02-06 16:01:00,00:00:53,NaN,NaT,2025-03-05 14:32:00,2025-03-03 10:54:00,51.0,2025-03-03 16:11:00,False,Archimède KESSI,NaN,35888242369,533123940549,14764264148,NaN,0.000000,00:00:00,S2025-10,2025-03-03,S2025-03,0,1
3,101418180838,BEGUE - beguejf@hotmail.fr,Melinda Marmin,Jean Francois Begue (beguejf@hotmail.fr),ADV,Résolu,2025-03-05 10:51:00,2025-03-13 14:28:00,NaN,2025-03-13 14:26:00,195:36:46,195:35:03,NaN,BEGUE,NaN,NaN,2025-03-13 14:26:00,NaN,"ADV (contrats, résiliation, commerce)",NaN,NaN,Suivi de ma commande,E-mail,10708730071,"Bonjour à tous,\nCi-joint un Complément de co...",Changer la phase de la transaction après la si...,NaN,NaN,NaN,NaN,NaN,beguejf@hotmail.fr,2025-02-05 14:28:00,195:36:46,NaN,NaT,2025-03-13 15:25:00,2025-03-13 14:26:00,37.0,2025-03-13 14:27:00,False,Melinda Marmin,NaN,2245,543589404905,86831900884,NaN,57

In [21]:
follow = df[(df['Pipeline'] == 'SPSA') 
            & (df['Formulaire SPSA'] == 'C2 - Assistance niveau 2') 
            & (df['Associated Company'] == 'FOLLOW') 
            & (df['Source'] != 'Migration JIRA')]

follow = follow[follow['Date de création'] > pd.to_datetime('2024-08-01')]

In [22]:
follow['Formulaire SPSA'].value_counts()

Formulaire SPSA
C2 - Assistance niveau 2    391
Name: count, dtype: int64

In [65]:
print(follow['Date de création'].min())
print(follow['Date de création'].max())

2024-08-01 10:43:00
2025-03-17 15:10:00


In [ ]:
import pandas as pd
import plotly.express as px

# Regroupement par semaine et calcul de la moyenne des heures travaillées
follow_groupe = follow.groupby('Semaine').agg({'working_hours': 'mean'}).reset_index()

# Filtrer uniquement les semaines ayant des valeurs (exclure NaN et 0 si nécessaire)
follow_groupe = follow_groupe[follow_groupe['working_hours'].notna()]

# Supprimer la semaine "S2024-01"
follow_groupe = follow_groupe[follow_groupe['Semaine'] != "S2024-01"]

# Création du graphique à barres
fig = px.bar(follow_groupe, x='Semaine', y='working_hours', title='Follow -Moyenne temps 1er réponse aux tickets')

# Ajout d'une ligne horizontale de référence à 2
fig.add_hline(y=2, line_dash="dot", line_color="red", annotation_text="Référence : 2h")

# Bornage de l'axe Y à 40
fig.update_layout(yaxis=dict(range=[0, 30]))

# Affichage du graphique
fig.show()


In [47]:
count_sup_2 = (follow['working_hours'] > 2).sum()
print("Nombre de fois où le temps de réponse a un ticket C2 est supérieur à 2 :", count_sup_2, "/", len(follow))

Nombre de fois où le temps de réponse a un ticket C2 est supérieur à 2 : 145 / 391


In [53]:
# Filtrer les semaines entre "S2024-31" et "S2024-43" incluses
follow_1er_partie = follow[(follow['Semaine'] >= "S2024-31") & (follow['Semaine'] <= "S2024-47")]

count_sup_2 = (follow_1er_partie['working_hours'] > 2).sum()
print("Nombre de fois où le temps de réponse a un ticket C2 est supérieur à 2 :", count_sup_2, "/", len(follow_1er_partie))

proportion = count_sup_2 / len(follow_1er_partie)
proportion


Nombre de fois où le temps de réponse a un ticket C2 est supérieur à 2 : 34 / 193


0.17616580310880828

In [54]:
# Filtrer les semaines entre "S2024-31" et "S2024-43" incluses
follow_2eme_partie = follow[(follow['Semaine'] >= "S2024-47")]

count_sup_2 = (follow_2eme_partie['working_hours'] > 2).sum()
print("Nombre de fois où le temps de réponse a un ticket C2 est supérieur à 2 :", count_sup_2, "/", len(follow_2eme_partie))

proportion = count_sup_2 / len(follow_2eme_partie)
proportion



Nombre de fois où le temps de réponse a un ticket C2 est supérieur à 2 : 114 / 201


0.5671641791044776

In [55]:
follow.working_hours.describe()

count    391.000000
mean       3.642455
std        8.209832
min        0.000000
25%        0.466667
50%        1.316667
75%        3.266667
max      101.533333
Name: working_hours, dtype: float64

In [56]:
follow_1er_partie.working_hours.describe()

count    193.000000
mean       1.945250
std        5.588426
min        0.000000
25%        0.366667
50%        0.783333
75%        1.683333
max       58.900000
Name: working_hours, dtype: float64

In [57]:
follow_2eme_partie.working_hours.describe()

count    201.000000
mean       5.089221
std        9.478601
min        0.000000
25%        0.683333
50%        2.200000
75%        5.650000
max      101.533333
Name: working_hours, dtype: float64

In [67]:
follow.Catégorie.value_counts()

Catégorie
Facturation                                            68
PbConnexion                                            41
PC/SC                                                  30
Information                                            30
Remplacant / Collaborateur                             22
Rejets                                                 18
Recettes                                               16
Authentification / sécurisation CPS                    15
Appairage                                              13
Stellair Connexion                                     13
ADV (contrats, résiliation, commerce)                  12
Téléservices                                            9
Fonctionnalités                                         9
Lecture CPS                                             8
SCOR                                                    6
Facturation C2S                                         6
Facturation TP AMC                                      6
Lect

In [17]:
follow.to_excel('follow.xlsx')

In [22]:
df.Source.value_counts(normalize=True)

Source
Chat              0.266295
Téléphone         0.265497
Formulaire        0.223640
E-mail            0.154475
Migration JIRA    0.090094
Name: proportion, dtype: float64

In [25]:
df[(df['Pipeline'] == 'SSIA') & ((df['Source'] == 'Chat') | (df['Source'] == 'E-mail'))].groupby('Mois')['Ticket ID'].nunique().reset_index()

,Mois,Ticket ID
0,S2024-05,1
1,S2024-06,122
2,S2024-07,135
3,S2024-08,97
4,S2024-09,155
5,S2024-10,210
6,S2024-11,169
7,S2024-12,271
8,S2025-01,238


In [9]:
df[df['Propriétaire du ticket'] == 'Pierre Goupillon'].groupby('Semaine')['Ticket ID'].nunique().reset_index()

,Semaine,Ticket ID
0,S2022-45,1
1,S2023-43,1
2,S2023-47,1
3,S2024-01,3
4,S2024-06,1
5,S2024-11,1
6,S2024-13,1
7,S2024-14,1
8,S2024-16,1
9,S2024-17,3


In [107]:
df.Source.value_counts()

Source
Chat              1336
Téléphone         1332
Formulaire        1122
E-mail             775
Migration JIRA     452
Name: count, dtype: int64

In [108]:
repartition_activite_agent(data_affid_hubspot_agent)

In [109]:
repartition_tickets_agent_semaine(df)

In [110]:
# Nombre de ticket / Semaine par Catégorie
def metrics_nombre_ticket_categorie(df, pipeline):
    df = df[df['Pipeline'] == pipeline].groupby('Catégorie')['Nombre_ticket_client'].sum().reset_index().sort_values(by='Nombre_ticket_client', ascending=False).head(20)
    return df

metrics_nombre_ticket_categorie(df, 'SSIA')

,Catégorie,Nombre_ticket_client
29,Facturation,240
72,Lecteur,169
63,Information,111
52,Fonctionnalités,79
2,"ADV (contrats, résiliation, commerce)",79
100,PC/SC,56
102,PbConnexion,54
68,Installation,53
106,Recettes,42
110,Remplacant / Collaborateur,41


In [111]:
activite_ticket_source_client(df)

In [112]:
df['Formulaire SPSA'].value_counts(normalize=True) * 100

Formulaire SPSA
C2 - Assistance niveau 2                                      46.488947
B2 - Mise à niveau partenaire avec support                     9.232770
A4 - Expédition PRIUM-4 achat                                  7.672302
C4 - Maintenance lecteur                                       6.762029
A1 - Expédition NEO achat                                      6.241873
B4 - Appairage multi-utilisateurs                              4.746424
B3 - Mise à niveau Olaqin                                      4.551365
A3 - Expédition NEO location Olaqin                            3.641092
B1 - Mise à niveau partenaire sans support                     2.730819
D2 - Désactivation/réactivation d'un compte utilisateur        2.535761
A5 - Expédition PRIUM-4 location Olaqin                        1.560468
A2 - Expédition NEO location partenaire                        1.495449
D1 - Résiliation location lecteur partenaire                   0.845254
A6 - Expédition iWL350 location Olaqin          

In [113]:
activite_ticket_source_pipeline(df)

In [114]:
# Nombre de ticket / Semaine par Pipeline
def metrics_nombre_ticket_pipeline(df, pipeline):
    df = df[df['Source'].isin(['Chat', 'E-mail'])]
    df = df[df['Pipeline'] == pipeline].groupby(['Propriétaire du ticket'])['Ticket ID'].nunique().reset_index().sort_values(by='Ticket ID', ascending=False)
    return df

metrics_nombre_ticket_pipeline(df, 'SSIA')

,Propriétaire du ticket,Ticket ID
5,FREDERIC SAUVAN,482
0,Archimède KESSI,344
2,Christophe Brichet,201
6,HUMBLOT NASSUF,94
11,Olivier Sainte-Rose,93
12,Pierre Goupillon,52
13,Sandrine Sauvage,35
7,Melinda Marmin,25
3,Emilie Gest,14
17,Vincent LE QUENNEC,14


In [ ]:
def sla_2h_spsa (df): 

    df_sla_2h = df[(df['working_hours'] < 2) 
                   & (df['working_hours'] > 0.1)].groupby(['Semaine'])['working_hours'].count().reset_index().rename(columns={'working_hours': 'SLA < 2h'})

    df_tickets = df.groupby(['Semaine'])['working_hours'].count().reset_index().rename(columns={'working_hours': 'Nb tickets'})
    

    df_tickets_std = df['working_hours'].describe().to_frame()


    df_tickets_sla = pd.merge(df_tickets, df_sla_2h, how='left', on='Semaine')

    pourcentage_sla = (df_tickets_sla['SLA < 2h'].mean() / df_tickets_sla['Nb tickets'].mean()) * 100
    pourcentage_sla = (f"{pourcentage_sla:.2f}%")
    return pourcentage_sla, df_tickets_std

In [55]:
df_filtre_partenaire = df[(df['Pipeline'] == 'SPSA')
                            & (df['Formulaire SPSA'] == 'C2 - Assistance niveau 2')
                            & (df['Associated Company'] == 'FOLLOW') 
                            & (~df['Source'].isin(['Migration JIRA']))]

In [56]:
sla_partenaire, describe_partenaire = sla_2h_spsa(df_filtre_partenaire)

In [57]:
sla_partenaire

'64.51%'

In [58]:
describe_partenaire

,working_hours
count,392.000000
mean,3.459991
std,7.402725
min,0.000000
25%,0.379167
50%,1.000000
75%,2.466667
max,58.900000


In [45]:
df[(df['Pipeline'] == 'SPSA')
                & (df['Formulaire SPSA'] == 'C2 - Assistance niveau 2')
                & (df['Associated Company'] == 'FOLLOW') 
                & (~df['Source'].isin(['Migration JIRA']))]['working_hours'].describe()

count    392.000000
mean       3.459991
std        7.402725
min        0.000000
25%        0.379167
50%        1.000000
75%        2.466667
max       58.900000
Name: working_hours, dtype: float64

In [116]:
# Temps moyen de fermeture des tickets
df_temps_fermeture = df[df['Statut du ticket'].isin(['Résolu', 'Clos'])]
df_temps_fermeture = df_temps_fermeture[~df_temps_fermeture['Source'].isin(['Migration JIRA'])]
#df_temps_fermeture['Délai de traitement (HH:mm:ss)'].mean()

In [117]:
import pandas as pd
# Nombre de tickets ouvert
def metrics_nombre_ticket_ouvert(df, pipeline): 
    
    df = df[df['Pipeline'] == pipeline]
    # Conversion des colonnes en format datetime
    df['Date de création'] = pd.to_datetime(df['Date de création'], errors='coerce')
    df['Date de fermeture'] = pd.to_datetime(df['Date de fermeture'], errors='coerce')

    # Gérer les valeurs manquantes
    df = df[df['Date de création'].notna()]  # Exclure les tickets sans date de création
    df['Date de fermeture'] = df['Date de fermeture'].fillna(datetime.now())

    # Étendre les semaines ouvertes pour chaque ticket
    def generate_week_range(row):
        start_date = row['Date de création']
        end_date = row['Date de fermeture']
        return pd.date_range(start=start_date, end=end_date, freq='W-MON').strftime('S%Y-%U').tolist()

    df['Semaines ouvertes'] = df.apply(generate_week_range, axis=1)

    # Étendre le DataFrame pour avoir une ligne par semaine
    df_expanded = df.explode('Semaines ouvertes')

    # Ajouter une colonne pour indiquer qu'un ticket est ouvert
    df_expanded['Ticket_ouvert'] = 1

    # Conserver l'ID du ticket sur toutes les lignes
    df_expanded = df_expanded[['Ticket ID', 'Semaines ouvertes', 'Ticket_ouvert']]

    # Compter les tickets ouverts par semaine avec la liste des IDs
    tickets_par_semaine = df_expanded.groupby('Semaines ouvertes').agg({
        'Ticket ID': 'nunique',
    }).reset_index()

    # Renommer les colonnes pour plus de clarté
    #tickets_par_semaine.rename(columns={
    #    'Semaines ouvertes': 'Semaine',
    ##    'Ticket_ouvert': 'Nombre de tickets',
    #}, inplace=True)


    return tickets_par_semaine


In [118]:
metrics_nombre_ticket_ouvert(df, 'SSI')

,Semaines ouvertes,Ticket ID
0,S2022-46,1
1,S2022-47,1
2,S2022-48,1
3,S2022-49,1
4,S2022-50,1
...,...,...
111,S2024-52,76
112,S2025-01,70
113,S2025-02,64
114,S2025-03,57


In [119]:
df[~df['Statut du ticket'].isin(['Clos', 'Résolu'])].Source.value_counts()

Source
E-mail            87
Formulaire        87
Chat              80
Téléphone         68
Migration JIRA     7
Name: count, dtype: int64

In [120]:
df.dtypes

Ticket ID                                                                            int64
Nom du ticket                                                                       object
Associated Contact                                                                  object
Pipeline                                                                            object
Statut du ticket                                                                    object
Date de création                                                            datetime64[ns]
Date de fermeture                                                           datetime64[ns]
Propriétaire du ticket                                                              object
Source                                                                              object
Date de la dernière activité                                                datetime64[ns]
Délai de traitement (HH:mm:ss)                                                      object

In [121]:
%pwd

'c:\\Users\\ChristopheBRICHET\\OneDrive - OLAQIN\\Python\\depot'

In [66]:
import joblib
import pandas as pd
#pd.options.plotting.backend = 'hvplot'
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

clf = joblib.load('Entrainement_modele\\random_forest_model.pkl')
tfidf = joblib.load('Entrainement_modele\\tfidf_vectorizer.pkl')

clf_v2 = joblib.load('Entrainement_modele\\random_forest_model_v2.pkl')
tfidf_v2 = joblib.load('Entrainement_modele\\tfidf_vectorizer_v2.pkl')

stop_words = stopwords.words('french')

def nombre_de_mots(description):
    if pd.isna(description):  # Vérifier si la description est NaN
        return 0  # Retourner 0 mots pour les NaN
    return len(description.split())

#Filtrer le DataFrame
df_filtre_chat = df[
    (df['Pipeline'].isin(['SSIA', 'SSI'])) &  # Filtrer sur Pipeline
    (df['Catégorie'].notna()) &              # Garder les lignes où Catégorie est non nulle
    (df['Description du ticket'].apply(nombre_de_mots) > 3) &
    ( df['Source'] == 'Chat') # Vérifier que la description a plus de 3 mots
    ]

df_filtre_formulaire = df[
    (df['Pipeline'].isin(['SPSA'])) &  # Filtrer sur Pipeline
    (df['Catégorie'].notna()) &              # Garder les lignes où Catégorie est non nulle
    (df['Description du ticket'].apply(nombre_de_mots) > 3) &
    ( df['Formulaire SPSA'] == 'C2 - Assistance niveau 2') # Vérifier que la description a plus de 3 mots
    ]

df_filtre = pd.concat([df_filtre_chat, df_filtre_formulaire])

def nettoyer_texte(texte):
    texte = texte.lower()  # Mettre en minuscule
    texte = re.sub(r'\W+', ' ', texte)  # Enlever la ponctuation
    mots = word_tokenize(texte)  # Tokenizer le texte
    mots = [mot for mot in mots if mot not in stop_words]  # Retirer les stopwords
    return mots

df_filtre['Description du ticket'] = df_filtre['Description du ticket'].fillna("")
df_filtre['Description du ticket'] = df_filtre['Description du ticket'].str.replace('_x000d_,', ' ', regex=True)

# Ajouter une colonne "Mots" avec les mots tokenisés
df_filtre['Mots'] = df_filtre['Description du ticket'].apply(nettoyer_texte)



In [67]:
# Transformer les nouvelles descriptions en vecteurs TF-IDF
X_nouveau = tfidf.transform(df_filtre['Description du ticket'])

# Définir un seuil minimal de confiance pour accepter la prédiction
seuil_confiance = 0.30  # Seuil de 30%

# Obtenir les probabilités de chaque classe
probas = clf.predict_proba(X_nouveau)

# Obtenir les classes associées au modèle
classes = clf.classes_

# Trouver la classe prédite et sa probabilité
predictions = []
for i, prob in enumerate(probas):
    max_proba = np.max(prob)  # Probabilité maximale prédite pour cet échantillon
    predicted_class = classes[np.argmax(prob)]  # Classe associée à la probabilité max
    
    # Si la probabilité est inférieure au seuil, on classe en "NC"
    if max_proba < seuil_confiance:
        predictions.append("NC")
    else:
        predictions.append(predicted_class)

# Ajouter les prédictions corrigées au DataFrame
df_filtre['Catégorie prédite'] = predictions

# Afficher les résultats
print(df_filtre[['Description du ticket', 'Catégorie prédite']])

                                  Description du ticket  \
50    En tant que collaboratrice, comment faire avec...   
51    le lecteur ne fonctionne plus. Il a perdu un c...   
65    Je n'arrive pas à faire passer les mutuelles e...   
102   Bonjour depuis début septembre je n'arrive pas...   
103   J'ai 2 collaboratrice, je peux voir le profil ...   
...                                                 ...   
6373  Bonjour, _x000D_\n_x000D_\nVoici un retour du ...   
6376  Bonjour_x000D_\n_x000D_\nJe rencontre toujours...   
6380  Bonjour,_x000D_\n_x000D_\nLors de la connexion...   
6383  Bonjour_x000D_\n_x000D_\nje suis sous windows ...   
6385  Bonjour,_x000D_\n_x000D_\nPouvez vous renuméro...   

               Catégorie prédite  
50    Remplacant / Collaborateur  
51                       Lecteur  
65               Fonctionnalités  
102              Fonctionnalités  
103   Remplacant / Collaborateur  
...                          ...  
6373                          NC  
6376       

In [69]:
# Transformer les nouvelles descriptions en vecteurs TF-IDF
X_nouveau_v2 = tfidf_v2.transform(df_filtre['Description du ticket'])

# Définir un seuil minimal de confiance pour accepter la prédiction
seuil_confiance = 0.30  # Seuil de 30%

# Obtenir les probabilités de chaque classe
probas = clf_v2.predict_proba(X_nouveau_v2)

# Obtenir les classes associées au modèle
classes = clf_v2.classes_

# Trouver la classe prédite et sa probabilité
predictions = []
for i, prob in enumerate(probas):
    max_proba = np.max(prob)  # Probabilité maximale prédite pour cet échantillon
    predicted_class = classes[np.argmax(prob)]  # Classe associée à la probabilité max
    
    # Si la probabilité est inférieure au seuil, on classe en "NC"
    if max_proba < seuil_confiance:
        predictions.append("NC")
    else:
        predictions.append(predicted_class)

# Ajouter les prédictions corrigées au DataFrame
df_filtre['Catégorie prédite n2'] = predictions

# Afficher les résultats
print(df_filtre[['Description du ticket', 'Catégorie prédite n2']])

                                  Description du ticket  \
50    En tant que collaboratrice, comment faire avec...   
51    le lecteur ne fonctionne plus. Il a perdu un c...   
65    Je n'arrive pas à faire passer les mutuelles e...   
102   Bonjour depuis début septembre je n'arrive pas...   
103   J'ai 2 collaboratrice, je peux voir le profil ...   
...                                                 ...   
6373  Bonjour, _x000D_\n_x000D_\nVoici un retour du ...   
6376  Bonjour_x000D_\n_x000D_\nJe rencontre toujours...   
6380  Bonjour,_x000D_\n_x000D_\nLors de la connexion...   
6383  Bonjour_x000D_\n_x000D_\nje suis sous windows ...   
6385  Bonjour,_x000D_\n_x000D_\nPouvez vous renuméro...   

            Catégorie prédite n2  
50    Remplacant / Collaborateur  
51                       Lecteur  
65               Fonctionnalités  
102              Fonctionnalités  
103   Remplacant / Collaborateur  
...                          ...  
6373                          NC  
6376       

In [72]:
df_filtre.groupby(['Catégorie prédite', 'Catégorie prédite n2'])['Ticket ID'].count().to_excel('df_filtre.xlsx')

In [139]:
df_filtre[['Ticket ID', 'Description du ticket', 'Catégorie', 'Formulaire SPSA', 'Catégorie prédite']].to_excel('test.xlsx')

In [96]:
def nombre_de_mots(description):
    if pd.isna(description):  # Vérifier si la description est NaN
        return 0  # Retourner 0 mots pour les NaN
    return len(description.split())

# Filtrer le DataFrame
df_filtre = df[
    (df['Pipeline'].isin(['SSIA', 'SSI'])) &  # Filtrer sur Pipeline
    (df['Catégorie'].notna()) &              # Garder les lignes où Catégorie est non nulle
    (df['Description du ticket'].apply(nombre_de_mots) > 3) &
    ( df['Source'] == 'Chat') # Vérifier que la description a plus de 3 mots
    ]

df_filtre = df_filtre[['Ticket ID' ,'Catégorie', 'Description du ticket']]

In [98]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter



In [ ]:
### Régression logistique

# Télécharger les ressources nécessaires de nltk
nltk.download('all')
nltk.download('punkt', download_dir='C:\\Users\\ChristopheBRICHET\\anaconda3\envs\\streamlitenv\\Lib\\site-packages\\nltk')
nltk.download('stopwords', download_dir='C:\\Users\\ChristopheBRICHET\\anaconda3\envs\\streamlitenv\\Lib\\site-packages\\nltk')
nltk.data.path.append('C:\\Users\\ChristopheBRICHET\\anaconda3\envs\\streamlitenv\\Lib\\site-packages\\nltk')


In [ ]:



nltk.download('punkt')
nltk.download('stopwords')

# Charger les stopwords français
stop_words = stopwords.words('french')

# Liste des stopwords en français
#stop_words = set(stopwords.words('french'))

# Fonction de nettoyage des descriptions
def nettoyer_texte(texte):
    texte = texte.lower()  # Mettre en minuscule
    texte = re.sub(r'\W+', ' ', texte)  # Enlever la ponctuation
    mots = word_tokenize(texte)  # Tokenizer le texte
    mots = [mot for mot in mots if mot not in stop_words]  # Retirer les stopwords
    return mots

df_filtre['Description du ticket'] = df_filtre['Description du ticket'].fillna("")
df_filtre['Description du ticket'] = df_filtre['Description du ticket'].str.replace('_x000d_,', ' ', regex=True)

# Ajouter une colonne "Mots" avec les mots tokenisés
df_filtre['Mots'] = df_filtre['Description du ticket'].apply(nettoyer_texte)

# Regrouper les mots par catégorie
grouped_words = df_filtre.groupby('Catégorie')['Mots'].sum()

# Fonction pour compter les mots par catégorie
def extraire_mots_cles(mots):
    compteur = Counter(mots)
    return compteur.most_common(10)  # Top 10 mots-clés

# Appliquer la fonction aux groupes
mots_cles_par_categorie = grouped_words.apply(extraire_mots_cles)

# Fonction de nettoyage des descriptions
#def nettoyer_texte(texte):
#    if pd.isna(texte):  # Gérer les valeurs NaN
#        return ""
#    texte = texte.lower()  # Mettre en minuscule
#    texte = re.sub(r'\W+', ' ', texte)  # Enlever la ponctuation
###    mots = word_tokenize(texte, language='french')  # Tokenizer le texte
#    mots = [mot for mot in mots if mot not in stop_words_french]  # Retirer les stopwords
#    return " ".join(mots)  # Retourner une chaîne nettoyée
#    #return mots

# Remplacer les valeurs NaN et nettoyer les descriptions

# Préparer les données
X = df_filtre['Description du ticket']  # Les descriptions nettoyées (features)
y = df_filtre['Catégorie']  # Les catégories (labels)

# Transformer les descriptions en vecteurs numériques (TF-IDF)
tfidf = TfidfVectorizer(stop_words=stop_words)
X_tfidf = tfidf.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Entraîner un modèle (Random Forest ici)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Évaluer le modèle
y_pred = clf.predict(X_test)
#print("Accuracy:", accuracy_score(y_test, y_pred))
#print("Classification Report:\n", classification_report(y_test, y_pred))

# Prédire les catégories pour l'ensemble du DataFrame
X_test_full = tfidf.transform(df_filtre['Description du ticket'])  # Transformer les descriptions nettoyées
df_filtre['Catégorie prédite'] = clf.predict(X_test_full)  # Ajouter les prédictions comme nouvelle colonne

# Afficher le DataFrame avec la colonne "Catégorie prédite"
#print(df_filtre[['Description du ticket', 'Description du ticket clean', 'Catégorie prédite']])

# Définir un seuil minimal de confiance pour accepter la prédiction
seuil_confiance = 0.30  # Seuil de 30%

# Obtenir les probabilités de chaque classe
probas = clf.predict_proba(X_nouveau)

# Obtenir les classes associées au modèle
classes = clf.classes_

# Trouver la classe prédite et sa probabilité
predictions = []
for i, prob in enumerate(probas):
    max_proba = np.max(prob)  # Probabilité maximale prédite pour cet échantillon
    predicted_class = classes[np.argmax(prob)]  # Classe associée à la probabilité max
    
    # Si la probabilité est inférieure au seuil, on classe en "NC"
    if max_proba < seuil_confiance:
        predictions.append("NC")
    else:
        predictions.append(predicted_class)

# Ajouter les prédictions corrigées au DataFrame
nouvelle_data['Catégorie prédite'] = predictions

# Afficher les résultats
print(nouvelle_data[['Description du ticket', 'Catégorie prédite']])



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\ChristopheBRICHET
[nltk_data]    |     \AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to C:\Users\ChristopheBRIC
[nltk_data]    |     HET\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to C:\
[nltk_data]    |     Users\ChristopheBRICHET\AppData\Roaming\nltk_data
[nltk_data]    |     ...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\ChristopheBRICHET\AppData\Roaming\nltk_d
[nltk_data]    |     ata...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package aver

In [100]:
import joblib

# Sauvegarder le modèle Random Forest et le TF-IDF
joblib.dump(clf, 'random_forest_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


# Charger le modèle Random Forest et le TF-IDF
clf_loaded = joblib.load('random_forest_model.pkl')
tfidf_loaded = joblib.load('tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ChristopheBRICHET\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ChristopheBRICHET\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.3413654618473896
Classification Report:
                                        precision    recall  f1-score   support

           Activation / Désactivation       0.00      0.00      0.00         1
                         Connexion 3G       0.00      0.00      0.00         4
                              Echange       0.00      0.00      0.00         2
                          Facturation       0.00      0.00      0.00         1
                   Facturation TP AMC       0.00      0.00      0.00         1
                      Fonctionnalités       0.00      0.00      0.00         0
                          Information       0.00      0.00      0.00         1
                              Lecteur       0.00      0.00      0.00         0
                          Lecture CPS       0.00      0.00      0.00         1
                                  MAJ       0.00      0.00      0.00         0
                          PbConnexion       0.00      0.00      0.00         

In [72]:
df_filtre

,Ticket ID,Catégorie,Description du ticket,Description du ticket clean,categories_split,Catégorie prédite
50,3832946149,Authentification / sécurisation CPS,"En tant que collaboratrice, comment faire avec...",tant collaboratrice comment faire seul compte ...,Authentification / sécurisation CPS,Authentification / sécurisation CPS
51,3767236027,Lecteur,le lecteur ne fonctionne plus. Il a perdu un c...,lecteur fonctionne plus a perdu composant ress...,Lecteur,Lecteur
65,18572738804,Facturation,Je n'arrive pas à faire passer les mutuelles e...,arrive faire passer mutuelles identifier cps a...,Facturation,Facturation
102,16645766368,Recettes; Facturation,Bonjour depuis début septembre je n'arrive pas...,bonjour depuis début septembre arrive transfér...,Recettes,Recettes
102,16645766368,Recettes; Facturation,Bonjour depuis début septembre je n'arrive pas...,bonjour depuis début septembre arrive transfér...,Facturation,Recettes
...,...,...,...,...,...,...
4900,3334679248,Facturation,Après avoir inséré la carte vitale et au momen...,après avoir inséré carte vitale moment commenc...,Facturation,Facturation
4901,3334345941,Lecture CV,la commande lire la carte vitale ne fonctionne...,commande lire carte vitale fonctionne,Lecture CV,Lecture CV
4907,3302262226,Recettes,"comment transferer mes recettes , feuille de s...",comment transferer recettes feuille soin vers ...,Recettes,Recettes
4912,3284946888,Lecteur,mon ingenico ne lit pas les cartes vitales et ...,ingenico lit cartes vitales carte cps connecte...,Lecteur,Lecteur


In [337]:
# Nouvelle description
nouvelle_description = ["Problème lors de l'accès à l'application après mise à jour."]
nouvelle_description_tfidf = tfidf.transform(nouvelle_description)  # Transformer en vecteur TF-IDF
prediction = clf.predict(nouvelle_description_tfidf)  # Prédire la catégorie
print("Catégorie prédite :", prediction[0])


Catégorie prédite : Lecteur
